In [50]:
import tensorflow as tf
from utils import dataset_util
from vatic import VaticData
import os
from scipy.misc import imread
import label_map_util
import cv2

In [40]:
B = VaticData('B2HighwayNight')

Meta data path: /root/data/data-B2HighwayNight/meta.json does not exist. Use Default meta data
{'test': {'start': None, 'sets': [1], 'end': None, 'stride': 30}, 'train': {'start': None, 'sets': [0], 'end': None, 'stride': 1}, 'format': 'jpg'}
Total: 3160 images


In [3]:
flags = tf.app.flags
#flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS

In [58]:

def create_tf_example(bboxes, img_info, category_name2id,class_mapper={}):
  # TODO(user): Populate the following variables from your example.
  

  height = img_info['height']
  width = img_info['width']
  filename = img_info['path']
  with tf.gfile.GFile(filename, 'rb') as fid:
    encoded_jpg = fid.read()
   
  image_format = img_info['format'] 
  
  

  xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
  xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
  ymins = [] # List of normalized coordinates in bounding box (1 per box)
  ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
  classes_text = [] # List of string class name of bounding box (1 per box)
  classes = [] # List of integer class id of bounding box (1 per box)
    
  for bbox_id, bbox in bboxes.items():
     if bbox['occluded'] == 0 and bbox['outside'] == 0:
            
        xmin = bbox['x1']
        xmax = bbox['x1'] + bbox['width']
        ymin = bbox['y1'] 
        ymax = bbox['y1'] + bbox['height']        
        class_text = class_mapper[bbox['label']] if bbox['label'] in class_mapper else bbox['label']
        class_id = category_name2id[class_text]
       
        xmins.append(xmin)
        xmaxs.append(xmax)
        ymins.append(ymin)
        ymaxs.append(ymax)    
        classes_text.append(str(class_text))        
        classes.append(class_id)


  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return tf_example

In [42]:
NUM_CLASSES = 90
PATH_TO_LABELS = "/root/object_detection/data/mscoco_label_map.pbtxt"
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [43]:
get_category_name2id = lambda category_index: {item['name']: item['id'] for item in category_index.values()}

category_name2id = get_category_name2id(category_index)


def get_diff_classes(vatic ,category_index):
    return list(vatic.class_set - set(category_name2id))
get_diff_classes(A, category_index)

[u'trailer-head', u'sedan/suv', u'van']

In [44]:
class_mapper = {u'trailer-head': "truck", u'sedan/suv': "car", u'van': "car"}

In [59]:
#Try to write to TF record
output_path = "./B2HighwayNight.record"
writer = tf.python_io.TFRecordWriter(output_path)
img_info = {}
#Read the first image
img = imread(A.image_path_at(0))

img_info["height"], img_info['width'], _ = img.shape
img_info["format"] = A.image_path_at(0).split('.')[-1]
for i in range(A.size):
    img_info['path'] = A.image_path_at(i)
  
    bboxes = A.bbox_from_index(img_info['path'])
    tf_record = create_tf_example(bboxes, img_info, category_name2id, class_mapper)
    writer.write(tf_record.SerializeToString())
writer.close()